In [2]:
!pip install -U langchain langchain-community langchain-openai faiss-cpu pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 6.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: faiss-cpu━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [pypdf]
    Found existing installation: faiss-cpu 1.13.0━━━━━━━━━━━━━ 1/6 [pypdf]
    Uninstalling faiss-cpu-1.13.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [pypdf]
      Successfully uninstalled faiss-cpu-1.13.0━━━━━━━━━━━━━━━ 1/6 [pypdf]
  Attempting uninstall: langchain-core0m━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [faiss-cpu]
    Found existing installation: langchain-core 1.1.0━━━━━━━━━ 2/6 [faiss-cpu]
    Uninstalling langchain-core-1.1.0:━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [faiss-cpu]
      Successfully uninstalled langchain-core-1.1.0━━━━━━━━━━━ 2/6 [faiss-cpu]
  Attempting uninstall: langchain-openai0m━━━━━━━━━━━━━━━━━━━ 3/6 [langchain-core]
    Found existing installation: langchain-openai 1.1.0━━━━━━━ 3/6 [langchain-core]
    Uninstalling langchain-openai-1.1.0:90m━━━━━━━━━━━━━━━━━━━ 3/6 [langchain-core]
      Successfully uninstalled langchain

In [6]:
!pip install -U langchain-text-splitters


  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0


In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

# -------------------------------
# 2. Load PDFs
# -------------------------------
hc_loader = PyPDFLoader("./Mock_High_Court_Judgment_v4.pdf")
sc_loader = PyPDFLoader("./Mock_Supreme_Court_Judgment_v4.pdf")

hc_docs = hc_loader.load()
sc_docs = sc_loader.load()

In [12]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

hc_chunks = text_splitter.split_documents(hc_docs)
sc_chunks = text_splitter.split_documents(sc_docs)

In [13]:
hc_chunks

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-01-09T14:58:22+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-01-09T14:58:22+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Mock_High_Court_Judgment_v4.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='Mock High Court Judgment (Writ Petition Challenging State\nAction)\nCourt: High Court of a State (India)\nDISCLAIMER: This is a fictional judgment created exclusively for Agentic AI, NLP, and\nlegal-reasoning training purposes.'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2026-01-09T14:58:22+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2026-01-09T14:58:22+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': './Mock_High_Court_Judgment_v4.pdf', 'to

In [14]:
embeddings = OpenAIEmbeddings()
sc_vectorstore = FAISS.from_documents(sc_chunks, embeddings)


sample_hc_chunk = hc_chunks[5]
similar_sc_chunks = sc_vectorstore.similarity_search(
    sample_hc_chunk.page_content, k=3
)

# Create LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)



In [15]:
prompt = f"""
Compare the following High Court reasoning with Supreme Court reasoning
and explain the legal difference.

High Court:
{sample_hc_chunk.page_content}

Supreme Court:
{similar_sc_chunks[0].page_content}
"""

response = llm.invoke(prompt)
print(response.content)


The reasoning provided by the High Court and the Supreme Court reflects a similar foundational principle regarding the doctrine of constitutional balance and the limits of judicial review. However, there are notable differences in the depth and scope of analysis between the two courts.

### High Court Reasoning:
1. **Focus on Judicial Review**: The High Court emphasizes the principle that judicial review should not substitute legislative or executive wisdom unless the action is manifestly arbitrary, illegal, or disproportionate. This indicates a more straightforward application of the doctrine without delving into broader implications or context.
2. **Limited Scope**: The reasoning appears to be more concise and does not engage in a detailed examination of previous decisions or the broader constitutional framework. It suggests a more limited scope of analysis, focusing primarily on the immediate legal question at hand.

### Supreme Court Reasoning:
1. **In-depth Analysis**: The Supreme